In [8]:
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import sa
import psycopg2
import random


In [ ]:
#Create database connection - only do this step if connection in arcgis pro not already established
arcpy.management.CreateDatabaseConnection(
    out_folder_path=r"C:\Users\KOlso\Documents\GIS5572",
    out_name="Test",
    database_platform="POSTGRESQL",
    instance="34.30.71.239",
    account_authentication="DATABASE_AUTH",
    username="postgres",
    password='',
    save_user_pass="SAVE_USERNAME",
    database="gis_data",
    schema="",
    version_type="TRANSACTIONAL",
    version="",
    date=None,
    auth_type="",
    project_id="",
    default_dataset="",
    refresh_token='',
    key_file=None,
    role="",
    warehouse="",
    advanced_options=""
)

## Connect to databse - retrieve temperature data for Aug 31, 2024 - convert to fc

In [47]:
#probably going to delete this cell

# Set paths - using SDE files to connect to database
sde_path = r"G:\ArcGIS\Projects\5572\PostgreSQL-34-gis_data(postgres).sde"
source_table = f"{sde_path}\\gis_data.public.mesonet"
output_fc = r"G:\ArcGIS\Projects\5572\5572.gdb\mesonet_fc"

# Temporary feature class
arcpy.management.CreateFeatureclass(
    out_path=r"G:\ArcGIS\Projects\5572\5572.gdb",
    out_name="mesonet_fc",
    geometry_type="POINT",  # or POLYGON/LINE depending on your data
    spatial_reference=arcpy.Describe(source_table).spatialReference
)

# Add fields
arcpy.management.AddField(output_fc, "stid", "TEXT", field_length=50)
arcpy.management.AddField(output_fc, "temp_20240831", "DOUBLE")
arcpy.management.AddField(output_fc, "name", "TEXT", field_length=100)

# Fields to read from source and write to destination
read_fields = ["stid", "20240831", "name", "SHAPE@"]  # SHAPE@ gets geometry
write_fields = ["stid", "temp_20240831", "name", "SHAPE@"]

# Insert data
with arcpy.da.SearchCursor(source_table, read_fields) as search_cursor, \
     arcpy.da.InsertCursor(output_fc, write_fields) as insert_cursor:
    for row in search_cursor:
        # Optionally rename or validate data
        stid, temp, name, shape = row
        insert_cursor.insertRow((stid, temp, name, shape))

print(f"✅ Feature class created: {output_fc}")

✅ Feature class created: G:\ArcGIS\Projects\5572\5572.gdb\mesonet_fc


In [3]:
#connect to db
conn = psycopg2.connect(
    dbname="gis_data",
    user="postgres",
    password="heat2025risk",
    host="34.30.71.239",
    port="5432"
)

cur = conn.cursor()
cur.execute("""
    SELECT stid, "20240831", name, ST_AsText(geometry) 
    FROM gis_data.public.mesonet
""")
rows = cur.fetchall()
conn.close()

# --- Step 2: Prepare new feature class ---
output_fc = r"C:\Users\KOlso\OneDrive\Documents\ArcGIS\Projects\Test_GIS5572\Test_GIS5572.gdb\mesonet_fc_alt"
sr = arcpy.SpatialReference(4326)  # Adjust if your data uses a different SRID

# Create the feature class
arcpy.management.CreateFeatureclass(
    out_path=r"C:\Users\KOlso\OneDrive\Documents\ArcGIS\Projects\Test_GIS5572\Test_GIS5572.gdb",
    out_name="mesonet_fc_alt",
    geometry_type="POINT",
    spatial_reference=sr
)

# Add fields to the feature class
arcpy.management.AddField(output_fc, "stid", "TEXT", field_length=50)
arcpy.management.AddField(output_fc, "temp_20240831", "DOUBLE")
arcpy.management.AddField(output_fc, "name", "TEXT", field_length=100)

# --- Step 3: Insert features ---
with arcpy.da.InsertCursor(output_fc, ["stid", "temp_20240831", "name", "SHAPE@XY"]) as cursor:
    for stid, temp, name, geom_wkt in rows:
        try:
            # Convert WKT to geometry using arcpy
            geom = arcpy.FromWKT(geom_wkt)  # Convert WKT to geometry
            cursor.insertRow((stid, temp, name, (geom.centroid.X, geom.centroid.Y)))  # Use centroid for point
        except Exception as e:
            print(f"Error processing WKT for {stid}: {e}")

print("Feature class created from psycopg2 data.")

Feature class created from psycopg2 data.


## Interpolate with all points

In [4]:
#inerpolation 1 IDW
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\KOlso\OneDrive\Documents\ArcGIS\Projects\Test_GIS5572\Test_GIS5572.gdb"):
    out_raster = arcpy.sa.Idw(
        in_point_features="mesonet_fc_alt",
        z_field="temp_20240831",
        cell_size=0.0005,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
    out_raster.save(r"C:\Users\KOlso\OneDrive\Documents\ArcGIS\Projects\Test_GIS5572\Test_GIS5572.gdb\mesonet_alt_IDW_all")




In [5]:
#interpolation 2 - Ordinary Kriging

with arcpy.EnvManager(scratchWorkspace=r"C:\Users\KOlso\OneDrive\Documents\ArcGIS\Projects\Test_GIS5572\Test_GIS5572.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="mesonet_fc_alt",
        z_field="temp_20240831",
        kriging_model="Spherical # # # #",
        cell_size=0.0005,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\KOlso\OneDrive\Documents\ArcGIS\Projects\Test_GIS5572\Test_GIS5572.gdb\mesonet_alt_okri_all")


In [6]:
#interpolation #3 - Universal Kriging

with arcpy.EnvManager(scratchWorkspace=r"C:\Users\KOlso\OneDrive\Documents\ArcGIS\Projects\Test_GIS5572\Test_GIS5572.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="mesonet_fc_alt",
        z_field="temp_20240831",
        kriging_model="LinearDrift 0.000500 # # #",
        cell_size=0.0005,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\KOlso\OneDrive\Documents\ArcGIS\Projects\Test_GIS5572\Test_GIS5572.gdb\mesonet_alt_ukri_all")

In [1]:
#skip on laptop

#accuracy assessment - one approach
arcpy.ga.ExploratoryInterpolation(
    in_features="mesonet_fc_alt",
    value_field="temp_20240831",
    out_cv_table=r"G:\ArcGIS\Projects\5572\5572.gdb\Temp_0831_ExpInt_table",
    out_geostat_layer="test_output_ex_int",
    interp_methods="ORDINARY_KRIGING;UNIVERSAL_KRIGING;IDW",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)

<Result 'G:\\ArcGIS\\Projects\\5572\\5572.gdb\\Temp_0831_ExpInt_table'>

In [ ]:
#Ordinary Kriging shows as the best from this group

In [9]:
#Accuracy assessment, another approach that allows us to know for sure we're evaluating specific interpolation outputs

#but it doesn't actually work in the notebook for some reason - works in the GUI! 
# arcpy.ga.CompareGeostatisticalLayers(
#     in_geostat_layers="Mesonet_alt_GPI;Mesonet_alt_EKB;Mesonet_alt_IDW",
#     out_cv_table=r"G:\ArcGIS\Projects\5572\5572.gdb\Mesonet_0831_Validation_table",
#     out_geostat_layer=None,
#     comparison_method="SINGLE",
#     criterion="ACCURACY",
#     criteria_hierarchy="ACCURACY PERCENT #",
#     weighted_criteria="ACCURACY 1",
#     exclusion_criteria=None
# )


<class 'arcgisscripting.ExecuteError'>: Failed to execute. Parameters are not valid.
ERROR 000840: The value is not a Geostatistical Layer.
Failed to execute (CompareGeostatisticalLayers).


## Validate results

In [9]:
# Set environment
arcpy.env.workspace = r"C:\Users\KOlso\OneDrive\Documents\ArcGIS\Projects\Test_GIS5572\Test_GIS5572.gdb"
arcpy.env.overwriteOutput = True

# Inputs
input_fc = "mesonet_fc_alt"
z_field = "temp_20240831"
id_field = "stid"
cell_size = .0005  # Adjust based on your needs
num_folds = 9
fold_size = 3
output_table_base = "cv_results"  # Base name for all output tables

# Set extent to match the feature class
extent = arcpy.Describe(input_fc).extent
arcpy.env.extent = extent

# Create list of all OIDs and assign to folds
oids = [row[0] for row in arcpy.da.SearchCursor(input_fc, ["OID@"])]
random.shuffle(oids)
folds = [oids[i:i+fold_size] for i in range(0, len(oids), fold_size)]

# Create a new table for validation results (this will be recreated for each method)
def create_validation_table(output_table_name):
    """Creates a validation table with MAE and difference columns"""
    if arcpy.Exists(output_table_name):
        arcpy.management.Delete(output_table_name)
    arcpy.management.CreateTable(arcpy.env.workspace, output_table_name)
    arcpy.management.AddField(output_table_name, "Fold", "SHORT")
    arcpy.management.AddField(output_table_name, "OID", "LONG")
    arcpy.management.AddField(output_table_name, "stid", "TEXT", field_length=50)
    arcpy.management.AddField(output_table_name, "Actual", "DOUBLE")
    arcpy.management.AddField(output_table_name, "Predicted", "DOUBLE")
    arcpy.management.AddField(output_table_name, "Difference", "DOUBLE")
    arcpy.management.AddField(output_table_name, "MAE", "DOUBLE")  # This will store the MAE for the fold

### IDW cross validation

In [15]:
# Set the output table name for IDW results
output_table_idw = f"{output_table_base}_IDW"

# Create a table to store results for IDW
create_validation_table(output_table_idw)

# Perform IDW cross-validation
for i, test_oids in enumerate(folds):
    print(f"Processing fold {i+1}...")

    # Create training and testing feature layers
    train_layer = f"train_{i}"
    test_layer = f"test_{i}"
    test_oid_str = ",".join(str(oid) for oid in test_oids)
    train_where = f"OBJECTID NOT IN ({test_oid_str})"
    test_where = f"OBJECTID IN ({test_oid_str})"

    arcpy.management.MakeFeatureLayer(input_fc, train_layer, train_where)
    arcpy.management.MakeFeatureLayer(input_fc, test_layer, test_where)

    # Run IDW interpolation
    idw_raster = arcpy.sa.Idw(
        in_point_features=train_layer,
        z_field=z_field,
        cell_size=cell_size
    )

    idw_raster_name = f"idw_fold_{i+1}"
    idw_raster.save(idw_raster_name)

    # Extract predicted values to test points
    test_with_prediction = f"test_with_pred_{i}"
    arcpy.sa.ExtractValuesToPoints(test_layer, idw_raster, test_with_prediction, interpolate_values="NONE")

    # Record actual vs predicted
    fields = ["Fold", "OID@", id_field, z_field, "RASTERVALU"]
    with arcpy.da.SearchCursor(test_with_prediction, ["OID@", id_field, z_field, "RASTERVALU"]) as cursor_in, \
         arcpy.da.InsertCursor(output_table_idw, ["Fold", "OID", "stid", "Actual", "Predicted", "Difference", "MAE"]) as cursor_out:
        for row in cursor_in:
            actual_value = row[2]
            predicted_value = row[3]
            
            # Check if both actual and predicted values are not None
            if actual_value is not None and predicted_value is not None:
                diff = predicted_value - actual_value  # Difference = Predicted - Actual
                mae = abs(diff)  # MAE = absolute difference
                cursor_out.insertRow((i + 1, row[0], row[1], actual_value, predicted_value, diff, mae))
            else:
                # If either value is None, you can handle it by skipping or inserting a null value
                cursor_out.insertRow((i + 1, row[0], row[1], actual_value, predicted_value, None, None))


    # Clean up
    arcpy.management.Delete(train_layer)
    arcpy.management.Delete(test_layer)
    arcpy.management.Delete(test_with_prediction)
    arcpy.management.Delete(idw_raster)

print("✅ Cross-validation with IDW complete. Results saved to 'idw_cv_results'.")

Processing fold 1...
Processing fold 2...
Processing fold 3...
Processing fold 4...
Processing fold 5...
Processing fold 6...
Processing fold 7...
Processing fold 8...
Processing fold 9...
✅ Cross-validation with IDW complete. Results saved to 'idw_cv_results'.


In [14]:
print(f"Fold {i+1} - Test points: {arcpy.management.GetCount(test_with_prediction)[0]}")

Fold 9 - Test points: 3


### Ordinary Kriging Cross Validation

In [16]:
# Set the output table name for OK results
output_table_ok = f"{output_table_base}_OK"

# Create a table to store results for Ordinary Kriging
create_validation_table(output_table_ok)

# Perform Ordinary Kriging cross-validation
for i, test_oids in enumerate(folds):
    print(f"Processing fold {i+1}...")

    # Create training and testing feature layers
    train_layer = f"train_{i}"
    test_layer = f"test_{i}"
    test_oid_str = ",".join(str(oid) for oid in test_oids)
    train_where = f"OBJECTID NOT IN ({test_oid_str})"
    test_where = f"OBJECTID IN ({test_oid_str})"

    arcpy.management.MakeFeatureLayer(input_fc, train_layer, train_where)
    arcpy.management.MakeFeatureLayer(input_fc, test_layer, test_where)

    # Run Ordinary Kriging interpolation
    ok_result = arcpy.sa.Kriging(
        in_point_features=train_layer,
        z_field=z_field,
        kriging_model="Spherical",  # Change as needed
        cell_size=cell_size,
        search_radius="VARIABLE 12"
    )

    ok_raster_name = f"ok_fold_{i+1}"
    ok_result.save(ok_raster_name)

    # Extract predicted values to test points
    test_with_prediction = f"test_with_pred_{i}"
    arcpy.sa.ExtractValuesToPoints(test_layer, ok_result, test_with_prediction, interpolate_values="NONE")

    # Record actual vs predicted
    fields = ["Fold", "OID@", id_field, z_field, "RASTERVALU"]
    with arcpy.da.SearchCursor(test_with_prediction, ["OID@", id_field, z_field, "RASTERVALU"]) as cursor_in, \
         arcpy.da.InsertCursor(output_table_ok, ["Fold", "OID", "stid", "Actual", "Predicted", "Difference", "MAE"]) as cursor_out:
        for row in cursor_in:
            actual_value = row[2]
            predicted_value = row[3]
            
            # Check if both actual and predicted values are not None
            if actual_value is not None and predicted_value is not None:
                diff = predicted_value - actual_value  # Difference = Predicted - Actual
                mae = abs(diff)  # MAE = absolute difference
                cursor_out.insertRow((i + 1, row[0], row[1], actual_value, predicted_value, diff, mae))
            else:
                # If either value is None, you can handle it by skipping or inserting a null value
                cursor_out.insertRow((i + 1, row[0], row[1], actual_value, predicted_value, None, None))

    # Clean up
    arcpy.management.Delete(train_layer)
    arcpy.management.Delete(test_layer)
    arcpy.management.Delete(test_with_prediction)
    arcpy.management.Delete(ok_result)

print("✅ Cross-validation with Ordinary Kriging complete. Results saved to 'ok_cv_results'.")

Processing fold 1...
Processing fold 2...
Processing fold 3...
Processing fold 4...
Processing fold 5...
Processing fold 6...
Processing fold 7...
Processing fold 8...
Processing fold 9...
✅ Cross-validation with Ordinary Kriging complete. Results saved to 'ok_cv_results'.


### Universal Kriging Cross Validation

In [17]:
# Set the output table name for UK results
output_table_uk = f"{output_table_base}_UK"

# Create a table to store results for Universal Kriging
create_validation_table(output_table_uk)

# Perform Universal Kriging cross-validation
for i, test_oids in enumerate(folds):
    print(f"Processing fold {i+1}...")

    # Create training and testing feature layers
    train_layer = f"train_{i}"
    test_layer = f"test_{i}"
    test_oid_str = ",".join(str(oid) for oid in test_oids)
    train_where = f"OBJECTID NOT IN ({test_oid_str})"
    test_where = f"OBJECTID IN ({test_oid_str})"

    arcpy.management.MakeFeatureLayer(input_fc, train_layer, train_where)
    arcpy.management.MakeFeatureLayer(input_fc, test_layer, test_where)

    # Run Universal Kriging interpolation
    uk_result = arcpy.sa.Kriging(
        in_point_features=train_layer,
        z_field=z_field,
        kriging_model="LinearDrift 0.000500 # # #",
        cell_size=cell_size,
        search_radius="VARIABLE 12",
    )

    uk_raster_name = f"uk_fold_{i+1}"
    uk_result.save(uk_raster_name)

    # Extract predicted values to test points
    test_with_prediction = f"test_with_pred_{i}"
    arcpy.sa.ExtractValuesToPoints(test_layer, uk_result, test_with_prediction, interpolate_values="NONE")

    # Record actual vs predicted
    fields = ["Fold", "OID@", id_field, z_field, "RASTERVALU"]
    with arcpy.da.SearchCursor(test_with_prediction, ["OID@", id_field, z_field, "RASTERVALU"]) as cursor_in, \
         arcpy.da.InsertCursor(output_table_uk, ["Fold", "OID", "stid", "Actual", "Predicted", "Difference", "MAE"]) as cursor_out:
        for row in cursor_in:
            actual_value = row[2]
            predicted_value = row[3]
            
            # Check if both actual and predicted values are not None
            if actual_value is not None and predicted_value is not None:
                diff = predicted_value - actual_value  # Difference = Predicted - Actual
                mae = abs(diff)  # MAE = absolute difference
                cursor_out.insertRow((i + 1, row[0], row[1], actual_value, predicted_value, diff, mae))
            else:
                # If either value is None, you can handle it by skipping or inserting a null value
                cursor_out.insertRow((i + 1, row[0], row[1], actual_value, predicted_value, None, None))
    # Clean up
    arcpy.management.Delete(train_layer)
    arcpy.management.Delete(test_layer)
    arcpy.management.Delete(test_with_prediction)
    arcpy.management.Delete(ok_result)
   


Processing fold 1...
Processing fold 2...
Processing fold 3...
Processing fold 4...
Processing fold 5...
Processing fold 6...
Processing fold 7...
Processing fold 8...
Processing fold 9...


In [ ]:
# THIS CELL IS A BUSTED VERSION BUT I'M USING IT FOR REFERENCE DON'T DELETE BUT DON'T RUN EITHER

results = []

for i in range(n_folds):
    train = [pt for j, fold in enumerate(folds) if j != i for pt in fold]
    test = folds[i]

    temp_fc = "in_memory/train"
    if arcpy.Exists(temp_fc):
        arcpy.management.Delete(temp_fc)
    arcpy.management.CreateFeatureclass("in_memory", "train", "POINT", spatial_reference=4326)
    for field in fields:
        arcpy.management.AddField(temp_fc, field, "TEXT" if field == id_field else "DOUBLE")
    with arcpy.da.InsertCursor(temp_fc, fields + ["SHAPE@"]) as cursor:
        for row in train:
            cursor.insertRow(row)

    uk_result = arcpy.sa.Kriging(
        in_point_features=temp_fc,
        z_field=z_field,
        kriging_model="LinearDrift 0.000500 # # #",
        cell_size=cell_size,
        search_radius="VARIABLE 12",
    )

    for row in test:
        shp = row[-1]
        stid = row[fields.index(id_field)]
        obs = row[fields.index(z_field)]
        pred = float(arcpy.GetCellValue_management(uk_result, shp.centroid).getOutput(0))
        diff = pred - obs
        mae = np.abs(diff)
        results.append((stid, obs, pred, diff, mae))

create_validation_table(f"{output_table_base}_UK", results)


In [18]:
#Calculate RMSE MAE for each method - print in notebook is fine

#pick best one
#upload that one to db, and flask app